# Obtenção de rating do contribuinte

Com o objetivo de direcionar o modelo que ditá quais dívidas são melhores de recuperar, será realizado um rating que envolva apenas as variáveis do contribuinte.

In [2]:
import os
import pandas as pd
import zipfile
from datetime import date
import dotenv
import boto3
from io import BytesIO
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings
warnings.filterwarnings("ignore")

rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

print("Iniciando carregamento dos dados")
zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)

def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3_resource.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())

base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')
# base_imovel = ler_bases_exportadas('imovel.csv')
# base_mercantil = ler_bases_exportadas('mercantil.csv')
base_notas_fiscais = ler_bases_exportadas('emissao_notas.csv')
#base_dim_datas = ler_bases_exportadas('dim_datas.csv')

#base_conjunta = pd.concat([base_imovel, base_mercantil])



Iniciando carregamento dos dados


In [3]:
print("Inicia transformação das variáveis sobre a dívida")

# Gera as variáveis de tempo
base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

base_conjunta.drop_duplicates(subset='cda', inplace=True) #Garantia que não houve duplicatas de linhas



Inicia transformação das variáveis sobre a dívida


In [6]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento', 'data_divida',
       'ano_inscricao_da'],
      dtype='object')

In [9]:
# Seleciona dados sobre a divida
dados_divida = base_conjunta[[ 'cda', 'id_contribuinte', 'tipo_divida', 'valor_tot', 'vlr_pago', 'protesto', 'ajuizamento', 'inscricao_divida', 'quantidade_reparcelamento', 'idade_divida']]
dados_divida.rename(columns={'id_contribuinte': 'id_pessoa'}, inplace=True)
dados_divida.dropna(subset=['id_pessoa'], inplace=True)
dados_divida['id_pessoa'] = dados_divida['id_pessoa'].astype(str) # persistindo tipo de dados
dados_divida

# # cria coluna e instancia como zero para preencher
# dados_divida['quantidade_reparcelamentos'] = 0

# # concateno as bases
# dados_divida = base_conjunta.merge(base_parcelas,how='left',on=['cda', 'tipo_divida', 'id_pessoa'],suffixes=('', '_PARC'))
# dados_divida.loc[ dados_divida['total_valor_pago'].isna(), 'total_valor_pago' ] = 0
# dados_divida

,cda,id_pessoa,tipo_divida,valor_tot,vlr_pago,protesto,ajuizamento,inscricao_divida,quantidade_reparcelamento,idade_divida
0,00000123c3d4731c6b3a8e268c80aedf1,ac28642d7c82b33f,imovel,2048.73,0.0,0,0,2022-09-07,0,1.0
1,00000123c3d4731c6b3a8e268c80aedf2,fc4b99b807fbed41,mercantil,2515.85,0.0,0,0,2022-12-30,0,1.0
2,00000b44c5ba1e669ceed47545e621dd2,96e8e553de69d7a4,mercantil,1278.97,0.0,0,0,2002-09-07,0,21.0
3,000014e359592e62d8a3e5cebc255ca6E,6dbe14da38a31dc1,mercantil,847.02,0.0,0,0,2023-06-15,0,0.0
4,0000331f601a73e52b46f673bf0c61251,870c08c252b25ad1,imovel,344.12,0.0,0,0,2012-10-17,0,11.0
...,...,...,...,...,...,...,...,...,...,...
2414893,ffffd8af5760cada9fcacff532cf50c41,d411f54ad97d1f39,imovel,1728.13,0.0,0,0,2019-06-28,0,4.0
2414894,ffffdb721b90282718565e26094f06ee2,51d6043a4efa1ed1,mercantil,240.77,0.0,0,0,2017-10-22,0,6.0
2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,9b512f210b460120,imovel,9390.43,0.0,0,1,1998-11-26,0,25.0
2414896,fffff192f894983c9ceed47545e621dd1,dd5538d2d38803a7,imovel,16594.80,0.0,0,1,2008-10-05,0,15.0


In [5]:
# # Somando valores a vista e aprcelados em nova coluna
# dados_divida['valor_pago_vista_parc'] = dados_divida['valor_pago'] + dados_divida['total_valor_pago']
# dados_divida['valor_pago_vista_parc'].sum() / 1000000

1343.1835638800003

In [ ]:
# valor_tot = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['valor_tot'].sum()
# valor_pago = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['vlr_pago'].sum()
# divida_protestada = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['protesto'].max()
# divida_ajuizada = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['ajuizamento'].max()
# ano_inscricao_da = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['inscricao_divida'].max()
# # quantidade_reparcelamentos = dados_divida.groupby(['cda', 'id_pessoa', 'tipo_divida'])['quantidade_reparcelamentos'].sum()

# aux = pd.merge(valor_tot, valor_pago, on = ['cda', 'id_pessoa', 'tipo_divida'], how = "left")
# aux2 = pd.merge(divida_protestada, divida_ajuizada, on = ['cda', 'id_pessoa', 'tipo_divida'], how = "left")
# # aux3 = pd.merge(ano_inscricao_da, on = ['cda', 'id_pessoa', 'tipo_divida'], how = "left")

# aux4 = pd.merge(aux, aux2, on = ['cda', 'id_pessoa', 'tipo_divida'], how = "left")
# aux5 = pd.merge(aux4, ano_inscricao_da, on = ['cda', 'id_pessoa', 'tipo_divida'], how = "left")


In [ ]:
# # renomeia a coluna criada para valor_pago usada no modelo
# aux5.rename( columns={'valor_pago_vista_parc':'valor_pago'}, inplace=True)
# aux5

In [ ]:
# # Calcula a idade da dívida ativa
# aux5['ano_atual'] = date.today().year
# aux5['anos_idade_da'] = aux5['ano_atual'] - aux5['ano_inscricao_da']
# dados_divida = aux5.drop(columns=['ano_atual'])
# dados_divida

In [10]:
# Trazendo as três chaves como colunas
dados_divida = dados_divida.reset_index()
dados_divida

,index,cda,id_pessoa,tipo_divida,valor_tot,vlr_pago,protesto,ajuizamento,inscricao_divida,quantidade_reparcelamento,idade_divida
0,0,00000123c3d4731c6b3a8e268c80aedf1,ac28642d7c82b33f,imovel,2048.73,0.0,0,0,2022-09-07,0,1.0
1,1,00000123c3d4731c6b3a8e268c80aedf2,fc4b99b807fbed41,mercantil,2515.85,0.0,0,0,2022-12-30,0,1.0
2,2,00000b44c5ba1e669ceed47545e621dd2,96e8e553de69d7a4,mercantil,1278.97,0.0,0,0,2002-09-07,0,21.0
3,3,000014e359592e62d8a3e5cebc255ca6E,6dbe14da38a31dc1,mercantil,847.02,0.0,0,0,2023-06-15,0,0.0
4,4,0000331f601a73e52b46f673bf0c61251,870c08c252b25ad1,imovel,344.12,0.0,0,0,2012-10-17,0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
2414893,2414893,ffffd8af5760cada9fcacff532cf50c41,d411f54ad97d1f39,imovel,1728.13,0.0,0,0,2019-06-28,0,4.0
2414894,2414894,ffffdb721b90282718565e26094f06ee2,51d6043a4efa1ed1,mercantil,240.77,0.0,0,0,2017-10-22,0,6.0
2414895,2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,9b512f210b460120,imovel,9390.43,0.0,0,1,1998-11-26,0,25.0
2414896,2414896,fffff192f894983c9ceed47545e621dd1,dd5538d2d38803a7,imovel,16594.80,0.0,0,1,2008-10-05,0,15.0


In [24]:
df = dados_divida.query("anos_idade_da < 10")
df

# Aqui temos os 56.475 cdas que estávamos jogando como pior pagador mapeados como parcelamento

,cda,id_pessoa,tipo_divida,valor_tot,valor_pago,protesto,divida_ajuizada,ano_inscricao_da,anos_idade_da
0,00000123c3d4731c6b3a8e268c80aedf1,ac28642d7c82b33f,imovel,2048.73,0.00,0,0,2022,1
1,00000123c3d4731c6b3a8e268c80aedf2,fc4b99b807fbed41,mercantil,2515.85,0.00,0,0,2022,1
3,000014e359592e62d8a3e5cebc255ca6E,6dbe14da38a31dc1,mercantil,847.02,0.00,0,0,2023,0
8,00005193f0c1f57c6b3a8e268c80aedf1,0bff42777c16d00c,imovel,424.62,0.00,0,0,2021,2
9,0000521b64bf28c8e52281a70553db461,b5df816e1786f0a3,imovel,490.83,0.00,0,0,2016,7
...,...,...,...,...,...,...,...,...,...
2170540,ffffafbbcfae89607c8fa7a67092eaab2,172160e5eb98bf3c,mercantil,2118.05,0.00,0,0,2016,7
2170544,ffffd120a8f3f2799fcacff532cf50c41,934532c6c448a899,imovel,802.85,0.00,0,0,2014,9
2170545,ffffd79d8aa612ac9fcacff532cf50c41,c6b16a1235d987d3,imovel,26104.42,26104.42,1,0,2022,1
2170546,ffffd8af5760cada9fcacff532cf50c41,d411f54ad97d1f39,imovel,1728.13,0.00,0,0,2019,4


In [11]:
dados_divida

,index,cda,id_pessoa,tipo_divida,valor_tot,vlr_pago,protesto,ajuizamento,inscricao_divida,quantidade_reparcelamento,idade_divida
0,0,00000123c3d4731c6b3a8e268c80aedf1,ac28642d7c82b33f,imovel,2048.73,0.0,0,0,2022-09-07,0,1.0
1,1,00000123c3d4731c6b3a8e268c80aedf2,fc4b99b807fbed41,mercantil,2515.85,0.0,0,0,2022-12-30,0,1.0
2,2,00000b44c5ba1e669ceed47545e621dd2,96e8e553de69d7a4,mercantil,1278.97,0.0,0,0,2002-09-07,0,21.0
3,3,000014e359592e62d8a3e5cebc255ca6E,6dbe14da38a31dc1,mercantil,847.02,0.0,0,0,2023-06-15,0,0.0
4,4,0000331f601a73e52b46f673bf0c61251,870c08c252b25ad1,imovel,344.12,0.0,0,0,2012-10-17,0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
2414893,2414893,ffffd8af5760cada9fcacff532cf50c41,d411f54ad97d1f39,imovel,1728.13,0.0,0,0,2019-06-28,0,4.0
2414894,2414894,ffffdb721b90282718565e26094f06ee2,51d6043a4efa1ed1,mercantil,240.77,0.0,0,0,2017-10-22,0,6.0
2414895,2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,9b512f210b460120,imovel,9390.43,0.0,0,1,1998-11-26,0,25.0
2414896,2414896,fffff192f894983c9ceed47545e621dd1,dd5538d2d38803a7,imovel,16594.80,0.0,0,1,2008-10-05,0,15.0


In [12]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da'
}
df_divida_ativa = dados_divida.rename(columns=colunas_nome)


# Criando variável target Y que será predita
df_divida_ativa['percentual_pago_cda'] = df_divida_ativa['vlr_pago'] / df_divida_ativa['valor_total_da']


print("Inicia a conexão com S3 para inscrição dos dados")
# Cria conexão ao s3 e preenche a tabela com os dados
s3_resource = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3_resource.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())

up_s3_files(dataframe=df_divida_ativa, 
            bucket_name=os.getenv("S3_BUCKET_NAME"),
            folder_name=os.getenv("S3_FOLDER_NAME"), 
            file_name='feature_store_divida.csv')

print("Dados atualizados e persistidos no bucket S3")
print("Processo finalizado")

Inicia a conexão com S3 para inscrição dos dados
Dados atualizados e persistidos no bucket S3
Processo finalizado
